Importing Libraries

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import math
import numpy as np
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score
import os
from pprint import pprint

Setting path variables and dataset size and number of abnormalities to be classified. Also change model_name to the name of the model that is being tested. The save_index is how often the model is going to be saved to the directory as a backup, save_index=n means the model is going to be saved every n epochs. 

In [2]:

model_path='/mnt/Velocity_Vault/ECG/Model/'
memmap_path='/mnt/Velocity_Vault/ECG/Dataset/'

disease_size=7
dataset_size=19653

save_index=3

loss_threshold=0.003
loss_counter_max=5

If the dataset has not been splitted and is going to be entirely used for testing set org=True, if it has been splitted change org=False, also remember to change the respective dataset sizes

In [ ]:
org=False

if org:
    signal_name=memmap_path+'ecg_signal'
    feature_name=memmap_path+'features'
    label_name=memmap_path+'labels'
else:
    signal_name=memmap_path+'train_signal'
    feature_name=memmap_path+'train_feat'
    label_name=memmap_path+'train_labels'

In [3]:
ecg_signal=np.memmap(signal_name, dtype='int16', mode='r', shape=(dataset_size, 12,5000))
features=np.memmap(feature_name, dtype='float32', mode='r', shape=(dataset_size, 12))
labels=np.memmap(label_name, dtype='int', mode='r', shape=(dataset_size, disease_size))


ecg_signal = torch.tensor(ecg_signal, dtype=torch.float32)  # Convert to float32
features = torch.tensor(features, dtype=torch.float32)      # Convert to float32
labels = torch.tensor(labels, dtype=torch.float32)          # Convert to float32 for multi-label classification


Code of the Model

In [4]:
# Define the Noam optimizer (as explained earlier)
class NoamOpt:
    def __init__(self, model, warmup_steps, factor=1, optimizer=None):
        self.model = model
        self.warmup_steps = warmup_steps
        self.factor = factor
        self.optimizer = optimizer if optimizer else optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.0)
        self.step_num = 0

    def rate(self):
        """Calculate the learning rate based on the Noam scheduler."""
        if self.step_num < self.warmup_steps:
            return self.factor * (self.step_num + 1) / self.warmup_steps
        else:
            return self.factor * (self.step_num + 1) ** -0.5

    def step(self):
        """Update the model's parameters."""
        self.step_num += 1
        lr = self.rate()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        self.optimizer.step()

    def zero_grad(self):
        """Clear gradients for the optimizer."""
        self.optimizer.zero_grad()


# Define your model architecture (CTN model and Transformer as already defined)
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].clone().detach().to(dtype=torch.float32, device=x.device)
        return self.dropout(x)

class Transformer(nn.Module):
    def __init__(self, d_model, nhead, d_ff, num_layers, dropout):
        super(Transformer, self).__init__()
        self.d_model = d_model
        self.h = nhead
        self.d_ff = d_ff
        self.num_layers = num_layers
        
        # Add dropout to the encoder layers
        self.pe = PositionalEncoding(d_model, dropout)

        # Transformer Encoder Layer
        encode_layer = nn.TransformerEncoderLayer(
            d_model=self.d_model, 
            nhead=self.h, 
            dim_feedforward=self.d_ff, 
            dropout=dropout,
            batch_first=True 
        )
        self.transformer_encoder = nn.TransformerEncoder(encode_layer, self.num_layers)

    def forward(self, x):
        # Permute dimensions to match Transformer expectations: (batch_size, seq_len, d_model)
        out = x.permute(0, 2, 1)
        out = self.pe(out)  # Add positional encoding
        out = out.permute(1, 0, 2)  # Permute back for transformer encoder
        out = self.transformer_encoder(out)
        out = out.mean(0)  # Global average pooling over sequence length
        return out


class CTN(nn.Module):
    def __init__(self, d_model, nhead, d_ff, num_layers, dropout_rate, deepfeat_sz, nb_feats, classes):
        super(CTN, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv1d(12, 128, kernel_size=14, stride=3, padding=2, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Conv1d(128, 256, kernel_size=14, stride=3, padding=0, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Conv1d(256, d_model, kernel_size=10, stride=2, padding=0, bias=False),
            nn.BatchNorm1d(d_model),
            nn.ReLU(inplace=True),
            nn.Conv1d(d_model, d_model, kernel_size=10, stride=2, padding=0, bias=False),
            nn.BatchNorm1d(d_model),
            nn.ReLU(inplace=True),
            nn.Conv1d(d_model, d_model, kernel_size=10, stride=1, padding=0, bias=False),
            nn.BatchNorm1d(d_model),
            nn.ReLU(inplace=True),
            nn.Conv1d(d_model, d_model, kernel_size=10, stride=1, padding=0, bias=False),
            nn.BatchNorm1d(d_model),
            nn.ReLU(inplace=True)
        )
        self.transformer = Transformer(d_model, nhead, d_ff, num_layers, dropout=0.1)
        self.fc1 = nn.Linear(d_model, deepfeat_sz)
        self.fc2 = nn.Linear(deepfeat_sz + nb_feats, len(classes))
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, wide_feats):
        z = self.encoder(x)  # Encoded sequence
        out = self.transformer(z)  # Transformer output
        out = self.dropout(F.relu(self.fc1(out)))
        out = self.fc2(torch.cat([wide_feats, out], dim=1))
        return out




Training the model. Change the number of epochs to desired size (50 works good in general). If too much resources are being used reduce the batch size in the DataLoader.

In [5]:

# Create DataLoader for training
train_dataset = TensorDataset(ecg_signal, features, labels)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)

# Hyperparameters
d_model = 256
nhead = 8
d_ff = 512
num_layers = 8
dropout_rate = 0.1
deepfeat_sz = 64
nb_feats = 12  # Update this to 12 to reflect the wide features
classes = [i for i in range(disease_size)] # 5 classes
warmup_steps = 1000  # Number of warm-up steps for the Noam optimizer
epochs = 50  # Number of training epochs

# Initialize model, loss function, and optimizer
model = CTN(d_model=d_model, nhead=nhead, d_ff=d_ff, num_layers=num_layers,
            dropout_rate=dropout_rate, deepfeat_sz=deepfeat_sz,
            nb_feats=nb_feats, classes=classes)

# Create Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.0)

# Use Noam optimizer
noam_optimizer = NoamOpt(model, warmup_steps, optimizer=optimizer)

# Loss function
criterion = nn.BCEWithLogitsLoss()



prev_loss=0
loss_counter=0

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    for batch in train_loader:
        ecg_signal, wide_feats, labels = batch
        optimizer.zero_grad()  # Zero gradients
        
        # Forward pass
        outputs = model(ecg_signal, wide_feats)
        
        # Compute loss
        loss = criterion(outputs, labels.float())
        loss.backward()  # Backpropagation
        
        # Step the Noam optimizer (updates learning rate and performs optimization)
        optimizer.step()

        running_loss += loss.item()
        
        # Calculate accuracy
        predicted = torch.sigmoid(outputs) > 0.5
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.numel()

    avg_loss = running_loss / len(train_loader)
    accuracy = correct_preds / total_preds * 100
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
        # Save the model after every 5 epochs
    if (epoch + 1) % save_index == 0:
        model_save_path = os.path.join(model_path, f"model_epoch_{epoch+1}.pth")
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved as model_epoch_{epoch+1}.pth")
        
    if (epoch + 1)==epochs:
        model_save_path = os.path.join(model_path, "final_model.pth")
        torch.save(model.state_dict(), model_save_path)
        print("Final Model Saved as final_model.pth")
        
    #print(avg)
    if abs(avg_loss-prev_loss)<loss_threshold:
        loss_counter+=1
    else:
        loss_counter=0
        
    prev_loss=avg_loss
        
    if loss_counter==loss_counter_max:
        print('Threshold reached')
        model_save_path = os.path.join(model_path, "final_model.pth")
        torch.save(model.state_dict(), model_save_path)
        print("Final Model Saved as final_model.pth")
        break



Epoch 1/50, Loss: 8.0942, Accuracy: 53.35%
Epoch 2/50, Loss: 2.2865, Accuracy: 70.20%
Epoch 3/50, Loss: 0.4602, Accuracy: 82.12%
Model saved as model_epoch_3.pth
Epoch 4/50, Loss: 0.3343, Accuracy: 86.71%
Epoch 5/50, Loss: 0.2776, Accuracy: 88.85%
Epoch 6/50, Loss: 0.2547, Accuracy: 89.74%
Model saved as model_epoch_6.pth
Epoch 7/50, Loss: 0.2400, Accuracy: 90.41%
Epoch 8/50, Loss: 0.2306, Accuracy: 90.73%
Epoch 9/50, Loss: 0.2197, Accuracy: 91.20%
Model saved as model_epoch_9.pth
Epoch 10/50, Loss: 0.2143, Accuracy: 91.41%
Epoch 11/50, Loss: 0.2072, Accuracy: 91.70%
Epoch 12/50, Loss: 0.2013, Accuracy: 91.98%
Model saved as model_epoch_12.pth
Epoch 13/50, Loss: 0.1939, Accuracy: 92.29%
Epoch 14/50, Loss: 0.1887, Accuracy: 92.48%
Epoch 15/50, Loss: 0.1826, Accuracy: 92.85%
Model saved as model_epoch_15.pth
Epoch 16/50, Loss: 0.1765, Accuracy: 93.11%
Epoch 17/50, Loss: 0.1697, Accuracy: 93.30%
Epoch 18/50, Loss: 0.1643, Accuracy: 93.56%
Model saved as model_epoch_18.pth
Epoch 19/50, Los